In [2]:
# !pip install py2neo

In [7]:
import os, requests, json
from py2neo import Graph

HOST = "neo4j"
neo4jauth = tuple(os.environ["NEO4J_AUTH"].split("/"))
graph = Graph(host=HOST, auth=(neo4jauth)) 

In [8]:
class_index = json.load(open('imagenet_class_index.json', 'r'))

In [15]:
labels = [ci[1] for ci in class_index.values()]

In [16]:
labels[0]

'tench'